# update_create_config
- creates `config.json`, or updates it, if necessary.
- Also, removes old `/cache`, `benchmark.obj`, `density.obj`, if config isn't recent enough


In [ ]:
from xarray import __version__ as xa_ver
from xclim import __version__ as xc_ver
from pandas import __version__ as pd_ver
from geopandas import __version__ as gp_ver
from joblib import __version__ as jl_ver

from pathlib import Path
import json

from joblib import Memory

In [ ]:
config_path = Path('config.json')
if not config_path.is_file():
    config = {}
else:
    with open(config_path,encoding='utf-8') as config_file:
        config = json.load(config_file)
    

### VERSIONS:
# check versions to delete or not the cache:
ver_new = {"xa":xa_ver,"xc":xc_ver,"pd":pd_ver,"gp":gp_ver,"jl":jl_ver}
if "versions" in config:
    ver_old = config["versions"]
    if any([((k not in ver_old) or (ver_old[k] != ver_new[k])) for k in ver_new]):
        # delete cache, benchmark.obj, etc.
        print("versions differ! deleting cached data.")
        !rm "./cache/*"
        !rm "benchmark.obj"
        !rm "density.obj"
        
config["versions"] = ver_new

### GLOBAL OPTS:
# Projection
biasadjust = 'single' # only changes the dsim url at the moment.
# Random city on load
init_rand_city = True
# dask threading:
dask_schedule = 'single-threaded'

config["options"] = dict(init_rand_city=init_rand_city,biasadjust=biasadjust, dask_schedule=dask_schedule)

### CSS:
template_css = """
.tg-excellent button {
    background-color: #90EE90 !important;
}
.tg-good button {
    background-color: #FFFFE0 !important;
}
.tg-average button {
    background-color: #FFE4C4 !important;
}
.tg-poor button {
    background-color: #F08080 !important;
}
.tgx button {
    border-radius: 30px !important;
}
.tgx .bk-btn-default.bk-active {
    border-color: #00f !important;
}
td {
    text-align: right;
} 
tr:nth-child(even) {
    background-color: #EEE;
}"""

config["css"] = template_css.replace('\n','\\n')

### URLS:
dref = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/ouranos/spatial-analogs/era5-land.ncml'
dsim = f'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/ouranos/spatial-analogs/cmip6_{biasadjust}.ncml'
places = ("https://pavics.ouranos.ca/geoserver/public/ows?" +
     "service=wfs&version=2.0.0&request=GetFeature" +
    "&typeName=public:ne_10m_populated_places&bbox=22,-170,83,-48" +
    "&propertyname=NAME,ADM0_A3,ADM1NAME,the_geom&outputFormat=application/json")
masks = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/ouranos/spatial-analogs/masks.nc'
benchmark = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/ouranos/spatial-analogs/benchmarks.nc'

config["url"] = dict(dref=dref,dsim=dsim,places=places,masks=masks,benchmark=benchmark)

with open(config_path,'w',encoding='utf-8') as config_file:
    json.dump(config,config_file, ensure_ascii=False, indent=4)
